In [1]:
%cd ~/NetBeansProjects/ExpLosion/
from notebooks.common_imports import *

/Volumes/LocalDataHD/m/mm/mmb28/NetBeansProjects/ExpLosion


# Does the classification evaluation framework work?
What is the effect of adding uniform random noise to vectors? Can our experiments tell between a set of vectors and a corrupted version of the same vectors? To find out, add uniform random noise from $-n$ to $n$ (x axis) to all non-zero entries in a set of `word2vec` embeddings.
     

In [6]:
d = {'vectors__unlabelled_percentage': 100,
         'labelled': 'amazon_grouped-tagged',
         'vectors__dimensionality': 100.0,
         'decode_handler': 'SignifiedOnlyFeatureHandler',
         'vectors__composer': 'Add',
         'vectors__rep': 0, 
         'k': 3,
         'vectors__unlabelled': 'wiki',
         'vectors__algorithm': 'word2vec',
         'document_features_tr': 'J+N+AN+NN',
         'document_features_ev': 'AN+NN'}
e = Experiment.objects.filter(**d).order_by('noise').values_list('id', flat=True)
from thesisgenerator.utils.output_utils import get_scores, get_cv_fold_count
import numpy as np
cv_folds = get_cv_fold_count(e)
scores, folds, _ = get_scores(e)
raw_names = ['%1.1f'%x for x in Experiment.objects.filter(id__in=e).order_by('noise').values_list('noise', flat=True)]

# calculate significance
from itertools import tee
from gui.user_code import get_demsar_params
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

print('ids', e, 'names', raw_names)
for i, (e1, e2) in enumerate(pairwise(e)):
    t, _, _ = get_demsar_params([e1, e2], name_format=['id'])
    if t.significant[0] == 'True':
        # significant difference exists
        raw_names[i] += '*'
#     else:
#         raw_names[i] += '   . '

names = np.repeat(raw_names, cv_folds)
df = pd.DataFrame(dict(Noise=names, F1=scores, folds=folds))
ax = sns.factorplot('Noise', 'F1', data=df, kind='bar', aspect=1.5);
ax.set_xticklabels(rotation=45);

# random baseline for comparison
rand = Experiment.objects.get(vectors__algorithm='random_vect', labelled='amazon_grouped-tagged').id
score, score_std = Results.objects.get(id=rand, 
                                          classifier='MultinomialNB').get_performance_info('macrof1')
plt.hlines(score, -100, 100)
plt.savefig('plot-noise-gigaword-add.pdf', format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.1)

ids [114, 174, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99] names ['0.0', '0.0', '0.2', '0.4', '0.6', '0.8', '1.0', '1.2', '1.4', '1.6', '1.8', '2.0']
running performance query for experiments [114, 174]
Missing results entry for exp 114 and classifier MultinomialNB
Missing results entry for exp 174 and classifier MultinomialNB


AttributeError: 'NoneType' object has no attribute 'significant'